In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
historical_defects = pd.read_csv('data/historical_defects.csv')
inspection_sessions = pd.read_csv('data/inspection_sessions.csv')
defects = pd.read_csv('data/defects.csv')
train_defects = pd.read_csv('data/train_defects.csv', delimiter=',')

In [3]:
historical_defects.columns, inspection_sessions.columns, defects.columns, train_defects.columns

(Index(['inspection_id', 'defect_id', 'defect_sequence_no',
        'repeat_defect_detected'],
       dtype='object'),
 Index(['inspection_id', 'machine_id', 'eval_set', 'inspection_number',
        'inspection_dow', 'inspection_hour_of_day',
        'days_since_last_inspection'],
       dtype='object'),
 Index(['defect_id', 'defect_subcategory_id', 'defect_category_id'], dtype='object'),
 Index(['inspection_id', 'defect_id', 'defect_sequence_no',
        'repeat_defect_detected'],
       dtype='object'))

In [4]:
# UNSUCCESSFUL attempt to correlate machine_id to defect_id
unique_defect_machine = train_defects.merge(inspection_sessions, on="inspection_id").groupby(['defect_id','machine_id'])['inspection_id'].count().reset_index()
unique_defect_machine[unique_defect_machine['inspection_id']!=1]


,defect_id,machine_id,inspection_id


In [5]:
# We are looking for potential indicators of repeat defect

# ~features describing the time of defect detected/not
# inspection_hour_of_day (accuracy may be influenced by time of day, fatigue, lighting)
# inspection_dow day of week

# ~features describing the condition of machine and the specific defect
# defect_sequence_no (the sequence/order of specific defect) {defect_id,sequence_no} -->category?
# machine_id
# days since last inspection (condition may have deteriorated much if inspected a long time ago)
# inspection number (how many times has it been inspected)

# ~features describing the defect itself
# not available in test set
# ?defect_id (doesn't the sub category and category combined imply the id?)
# defect_subcategory_id
# defect_category_id

In [6]:
train_data = train_defects.merge(defects, on="defect_id")
train_data = train_data.rename(columns = {'repeat_defect_detected':'label'})
train_data1 = train_data.merge(inspection_sessions, on='inspection_id')
# features = train_data1.drop(['label','eval_set','inspection_id','defect_id'], axis=1).columns


test_data = inspection_sessions[inspection_sessions['eval_set']=='test']
features = test_data.drop(['eval_set'], axis=1).columns

In [7]:
x_train = train_data1[features].values
y_train = train_data1['label'].values

x_test = test_data[features].values


In [ ]:
rf_model = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
rf_model.fit(x_train, y_train)
pred_train = rf_model.predict(x_train)

print(np.sqrt(mean_squared_error(y_train,pred_train)))
print(r2_score(y_train, pred_train))





In [ ]:


pred_test = rf_model.predict(x_test)
